In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


class LabelVectorizer(CountVectorizer):
    def __init__(self):
        super().__init__(stop_words=None)

    def fit(self, X, y=None):
        super().fit(''.join(X).split('|'))
        return self

In [ ]:
from sklearn.compose import ColumnTransformer

data_path = Path('dataset/sflln')

features = pd.read_csv(data_path / 'drug_features.csv', header=0)
  
ct = ColumnTransformer([
    ('target_vectorizer', LabelVectorizer(), 'TARGET'),
    ('enzyme_vectorizer', LabelVectorizer(), 'ENZYME'),
    ('pathway_vectorizer', LabelVectorizer(), 'PATH'),
])
X_trans = ct.fit_transform(features)  

with open(data_path / 'interactions.txt') as f:
    interactions = np.array(eval(f.read()))

In [ ]:
X_trans.todense().shape

In [ ]:
693	+ 150 + 279